In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

os.makedirs("/kaggle/temp/vision-verse/data/train")

In [ ]:
!pip install torch torchmetrics matplotlib seaborn imutils torchvision torchinfo

In [ ]:
#!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


%matplotlib inline
%config InlineBackend.figure_format='retina'
import warnings
warnings.filterwarnings("ignore")

import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms
from torchinfo import summary


#import torch_xla
#import torch_xla.core.xla_model as xm
#import torch_xla.debug.metrics as met
#import torch_xla.distributed.parallel_loader as pl
#import torch_xla.distributed.xla_multiprocessing as xmp
#import torch_xla.utils.utils as xu

# Preprocessing and Data Augmentation

In [ ]:

class DataPreprocess():
    
    def __init__(self):
        self.path = "/kaggle/input/vision-verse/"
        self.traindf = pd.read_csv(self.path+"data/train.csv")
        self.testdf = pd.read_csv(self.path+"data/test.csv")
    
    def labelEncoder(self):
        self.traindf["path"] = self.path+self.traindf["path"]
        self.encoder = LabelEncoder()
        self.traindf['label_enc'] = self.encoder.fit_transform(self.traindf['label'])
        return self.traindf
        
    def showImage(self, fileno, train):
        file = train.iloc[fileno,0]
        img = Image.open(file).convert("RGB")
        transform = transforms.Compose([transforms.Resize((299,299)),transforms.PILToTensor()])
        tensor_img = transform(img)
        grid = torchvision.utils.make_grid(tensor_img)
        plt.rcParams["figure.figsize"] = (20,5) 
        plt.axis('off')
        plt.imshow(grid.permute(1,2,0)) 
        width,height = img.size
        print("Width = {}".format(width))
        print("Height = {}".format(height))
        """return width,height"""
    
    def getTest(self):
        return self.testdf
    
    def splitTrain(self, val_split = 0.1): 
        tr,val = train_test_split(train,test_size = val_split, stratify = train['label_enc'])
        return tr,val

In [ ]:
class DataAugment():
    def __init__(self,train,encoder):
        self.path = "/kaggle/temp/vision-verse/data/train/"
        self.traindf = train
    def mirror(self):
        lst1 = []
        lst2 = []
        lst3 = []
        for i in range(self.traindf.shape[0]):
            src = self.traindf.iloc[i,0]
            label  = self.traindf.iloc[i,1]
            label_enc = self.traindf.iloc[i,2]
            img = Image.open(src).convert('RGB')
            img1 = img.transpose(Image.FLIP_LEFT_RIGHT)
            newpath = self.path+ str(label) +"/rev"+str(i)+".jpg"
            lst1.append(newpath)
            lst2.append(label)
            lst3.append(label_enc)
            img1.save(newpath)
        df = pd.DataFrame(list(zip(lst1,lst2,lst3)),columns = ['path','label','label_enc'])
        self.traindf = pd.concat([df,self.traindf],ignore_index = True)
        return self.traindf
    
    def crop(self):
        pass
        

In [ ]:
DF_datapreprocess = DataPreprocess()

train = DF_datapreprocess.labelEncoder()
test = DF_datapreprocess.getTest()

In [ ]:
for i in train.label.unique():
    os.mkdir("/kaggle/temp/vision-verse/data/train/"+i) #implement only once......otherwise error will happen

In [ ]:
data_aug = DataAugment(train,DF_datapreprocess.encoder)
train = data_aug.mirror()

In [ ]:
DF_datapreprocess.showImage(4000, train)

In [ ]:
train

In [ ]:
train["label"].value_counts()

In [ ]:
test

In [ ]:
test = DF_datapreprocess.getTest()
#test['path'] = '/kaggle/input/vision-verse/'+test['path']

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
#device = xm.xla_device()
#print(device)

In [ ]:
batch_size = 32
val_split = 0.2
shuffle = True
training_frame , validation_frame = DF_datapreprocess.splitTrain(val_split)

transform = transforms.Compose([
    transforms.Resize(size = (299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.485, 0.456, 0.406), 
        std=(0.229, 0.224, 0.225)) 
        ])
lr = 1e-4
maxepochs = 8


In [ ]:
train_sampler = SubsetRandomSampler(list(training_frame.index))
val_sampler = SubsetRandomSampler(list(validation_frame.index))

In [ ]:
class ImageDataset(Dataset):
    
    def __init__(self,df,transform):
        self.data = df
        #self.imgpath = self.data['path']
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        if torch.is_tensor(idx):
            idx = idx.to_list()
        
        image = self.transform(Image.open(self.data.iloc[idx,0].strip()).convert('RGB'))
        
        if 'label_enc' in self.data.columns:
            l = torch.Tensor([self.data.iloc[idx,2]]).long().squeeze()
            
            sample = {
                'id' : idx,
                'image' : image,
                'label' : l
            }
        else:
            sample = {
                'id': idx,
                'image': image
            }
            
        return sample 

In [ ]:
'''
train_dataset = ImageDataset(training_frame,transform)
val_dataset = ImageDataset(validation_frame,transform)
test_dataset = ImageDataset(test,transform) 

train_loader = torch.utils.data.DataLoader(train_dataset,batch_size,sampler = train_sampler)
'''

<h3>Self-Made Model 1 - Inception v3</h3>

In [ ]:
from torch import nn

class conv_block(nn.Module):
    
    def __init__(self,inc,outc,**kwargs):
        super().__init__()
        self.relu = nn.ReLU()
        self.conv = nn.Conv2d(inc,outc,**kwargs)
        self.batchnorm = nn.BatchNorm2d(outc)
    
    def forward(self,x):
        return self.relu(self.batchnorm(self.conv(x)))

In [ ]:
class ResNet(nn.Module):
    def __init__(self,module):
        super().__init__()
        self.module = module
        
    def forward(self,inp):
        return self.module(inp) + inp

In [ ]:
class Inception_block(nn.Module):
    
    def __init__(self,inc,out1,red3,out3,red5,out5,pool1):
        super().__init__()
        
        self.branch1 = conv_block(inc, out1, kernel_size = (1,1))
        
        self.branch2 = nn.Sequential(
                        conv_block(inc, red3, kernel_size = (1,1)),
                        conv_block(red3, out3, kernel_size = (3,3),padding = (1,1))
                        )
        
        self.branch3 = nn.Sequential(
                        conv_block(inc, red5, kernel_size = (1,1)),
                        conv_block(red5, out5, kernel_size = (5,5),padding = (2,2))
                        )
        
        self.branch4 = nn.Sequential(
                        nn.MaxPool2d(kernel_size = (3,3), stride = (1,1), padding = (1,1)),
                        conv_block(inc, pool1, kernel_size = (1,1))
                        )
    
    def forward(self,x):
        return torch.cat(
                [self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)],1
                )

In [ ]:
class FinalStage(nn.Module):
    
    def __init__(self,inc,out):
        super().__init__()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.7)
        self.conv = conv_block(inc, 128, kernel_size = 1)
        self.fc1 = nn.Linear(2048,1024)
        self.fc2 = nn.Linear(1024,512)
        self.resnet1 = ResNet(nn.Sequential(nn.Linear(512,512),nn.ReLU(),nn.Linear(512,512)))
        self.fc3 = nn.Linear(512,64)
        self.resnet2 = ResNet(nn.Sequential(nn.Linear(64,64),nn.ReLU(),nn.Linear(64,64)))
        self.fc4 = nn.Linear(64,out)
    
    def forward(self,x):
        x = self.conv(x)
        x = x.reshape(x.shape[0],-1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.resnet1(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.resnet2(x))
        x = self.fc4(x)
        return x

In [ ]:
class TheModel(torch.nn.Module):
    
    def __init__(self, fout = 1000):
        super().__init__()
        
        self.conv1 = conv_block(inc = 3, outc = 64, kernel_size = (7,7), stride = (2,2), padding = (3,3))
        self.maxpool1 = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        
        self.conv2 = conv_block(inc = 64, outc = 192, kernel_size = 3, stride = 1, padding = 1)
        self.maxpool2 = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        
        self.inception1a = Inception_block(192,64,96,128,16,32,32)
        self.inception1b = Inception_block(256,128,128,192,32,96,64)
        self.maxpool3 = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        
        self.inception2a = Inception_block(480, 192, 96, 208, 16, 48, 64)
        self.inception2b = Inception_block(512, 160, 112, 224, 24, 64, 64)
        self.inception2c = Inception_block(512, 128, 128, 256, 24, 64, 64)
        self.inception2d = Inception_block(512, 112, 144, 288, 32, 64, 64)
        self.inception2e = Inception_block(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.inception3a = Inception_block(832, 256, 160, 320, 32, 128, 128)
        self.inception3b = Inception_block(832, 384, 192, 384, 48, 128, 128)
        self.avgpool = nn.AvgPool2d(kernel_size=7, stride=1)
        
        self.dropout = nn.Dropout(p=0.4)
        self.final = FinalStage(1024, fout)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)

        x = self.inception1a(x)
        x = self.inception1b(x)
        x = self.maxpool3(x)

        x = self.inception2a(x)
        x = self.inception2b(x)
        x = self.inception2c(x)
        x = self.inception2d(x)
        x = self.inception2e(x)
        x = self.maxpool4(x)
        
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.avgpool(x)
        
        x = self.final(x)
        
        return x

In [ ]:
model = TheModel(4)
#print(model)
summary(model, input_size=(32, 3, 299, 299))

In [ ]:
err = torch.nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr)

In [ ]:
from tqdm import tqdm  #Useless Import 

def trainFunc(model, train_data, val_data, transformer, learning_rate, epochs):

    train = ImageDataset(train_data.copy(), transformer) 
    val = ImageDataset(val_data.copy(), transformer)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=32)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu") #bro, I already got the device

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr= learning_rate)

    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    for epoch_num in range(epochs):

        total_acc_train = 0
        total_loss_train = 0

        for train_input in tqdm(train_dataloader):

            train_label = train_input['label'].to(device)
            train_image = train_input['image'].to(device)

            output = model(train_image)
            
            batch_loss = criterion(output, train_label)
            total_loss_train += batch_loss.item()
            
            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
        
        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

            for val_input in val_dataloader:

                val_label = val_input['label'].to(device)
                val_image = val_input['image'].to(device)

                output = model(val_image)

                batch_loss = criterion(output, val_label)
                total_loss_val += batch_loss.item()
                
                acc = (output.argmax(dim=1) == val_label).sum().item()
                total_acc_val += acc
        
        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
            | Train Accuracy: {total_acc_train / len(train_data): .3f} \
            | Val Loss: {total_loss_val / len(val_data): .3f} \
            | Val Accuracy: {total_acc_val / len(val_data): .3f}')

In [ ]:
#all cool, I already set the hyperparams
EPOCHS = 32
model = TheModel()
LR = 1e-5
              
trainFunc(model = model, train_data = training_frame, val_data = validation_frame, transformer = transform, learning_rate = LR, epochs = EPOCHS)

In [ ]:
def evaluate(model, transformer, test_data):

    test = ImageDataset(test_data, transformer)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=32)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    
    output_list = []

    if use_cuda:
        model = model.cuda()

    with torch.no_grad():

        for test_input in test_dataloader:
            test_image = test_input['image'].to(device)
            
            output = model(test_image)
            
            output_list.append(output.argmax(dim=1))
        
    return output_list

In [ ]:
output = evaluate(model = model, transformer = transform, test_data = test)
print(output)

In [ ]:
output1 = output
test_out = []
for x in output:
    x = x.to("cpu")
    y = x.tolist()
    test_out.extend(y)

In [ ]:
print(len(test_out))

In [ ]:
test['label_enc'] = test_out

In [ ]:
test

In [ ]:
test['label']= DF_datapreprocess.encoder.inverse_transform(test['label_enc'])
test['path'] = test['path'].apply(
        lambda x: x.replace('/kaggle/input/vision-verse/','')        
        )
test.drop(['label_enc'],axis = 1,inplace = True)
test.head()

In [ ]:
#test['path'] = test['path'].apply(lambda x : x[1:])


In [ ]:
#test.drop(['path'],inplace = True, axis = 1)

In [ ]:
test

In [ ]:
test.to_csv('submission1.csv',index = False)

In [ ]:
t = pd.read_csv("/kaggle/input/vision-verse/data/test.csv")

In [ ]:
test = pd.concat([t,test],axis = 1,ignore_index = True)

In [ ]:
test